# Parcial 1 - MLG

In [1]:
library(Rfast)
library(Formula)
library(glmtoolbox)

Loading required package: Rcpp
Loading required package: RcppZiggurat


# Punto 1
Este conjunto de datos consta de las ventas de un producto en 200 mercados diferentes, junto con los presupuestos de publicidad del producto en cada uno de esos mercados para dos medios diferentes: televisión y radio. El objetivo del análisis consiste en explicar las diferencias entre los mercados con respecto a las ventas usando las diferencias entre los mismos en relación a sus presupuestos de publicidad en TV y radio.

In [2]:
# df = read.csv('https://www.statlearning.com/s/Advertising.csv')
data(advertising)
attach(advertising)

In [3]:
fit1 <- glm(sales ~ log(TV) + radio + log(TV)*radio, family=gaussian(log), data = advertising)
summary(fit1) # phi = 0.2282484


Call:
glm(formula = sales ~ log(TV) + radio + log(TV) * radio, family = gaussian(log), 
    data = advertising)

Deviance Residuals: 
     Min        1Q    Median        3Q       Max  
-1.29278  -0.35848   0.01735   0.33568   1.42959  

Coefficients:
                Estimate Std. Error t value Pr(>|t|)    
(Intercept)    1.0302249  0.0467959  22.015   <2e-16 ***
log(TV)        0.2646359  0.0092146  28.719   <2e-16 ***
radio         -0.0043242  0.0014929  -2.896   0.0042 ** 
log(TV):radio  0.0036644  0.0002909  12.598   <2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for gaussian family taken to be 0.2668851)

    Null deviance: 5417.149  on 199  degrees of freedom
Residual deviance:   52.309  on 196  degrees of freedom
AIC: 309.35

Number of Fisher Scoring iterations: 5


En la Normal tenemos que: $m_k = 1$ y $V(\mu_k) = 1$, luego el parámetro de dispersión es:
$$ \hat{\phi} = \frac{1}{n-p}\sum_{k = 1}^n (y_k -\hat{\mu}_k)^2$$

In [4]:
sum(residuals(fit1)^2)/(196) #phi

[1] 0.266885

In [5]:
fit2 <- glm(sales ~ log(TV) + radio + log(TV)*radio, family=gaussian(log), data = advertising[-c(3,90,6,127,76),])
summary(fit2) # phi = 0.2282484


Call:
glm(formula = sales ~ log(TV) + radio + log(TV) * radio, family = gaussian(log), 
    data = advertising[-c(3, 90, 6, 127, 76), ])

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-1.2287  -0.3522  -0.0036   0.3285   1.2600  

Coefficients:
                Estimate Std. Error t value Pr(>|t|)    
(Intercept)    1.0677403  0.0440050  24.264  < 2e-16 ***
log(TV)        0.2573868  0.0086527  29.747  < 2e-16 ***
radio         -0.0069010  0.0014951  -4.616 7.17e-06 ***
log(TV):radio  0.0041580  0.0002896  14.355  < 2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for gaussian family taken to be 0.2282484)

    Null deviance: 5255.313  on 194  degrees of freedom
Residual deviance:   43.595  on 191  degrees of freedom
AIC: 271.27

Number of Fisher Scoring iterations: 4


**B.** las ventas *medianas* dejando fija la inversión en publicidad en radio

In [6]:
phi = 0.2282484 
Q2_b1 = exp(fit2$coefficients[2]*log(1.2)+fit2$coefficients[4]*log(1.2)*15) + qnorm(0.5, mean = 0, sd = sqrt(phi))
paste(round((Q2_b1-1)*100,0), "%")

[1] "6 %"

In [7]:
Q2_b2 = exp(fit2$coefficients[2]*log(1.2)+fit2$coefficients[4]*log(1.2)*35) + qnorm(0.5, mean = 0, sd = sqrt(phi))
paste(round((Q2_b2-1)*100,1), "%")

[1] "7.6 %"

**C.** las ventas *medianas* dejando fija la inversión en publicidad en TV

In [8]:
Q2_c1 = exp(fit2$coefficients[3]*5+fit2$coefficients[4]*log(80)*5) + qnorm(0.5, mean = 0, sd = sqrt(phi))
paste(round((Q2_c1-1)*100,1), "%")

[1] "5.8 %"

In [9]:
Q2_c2 = exp(fit2$coefficients[3]*5+fit2$coefficients[4]*log(250)*5) + qnorm(0.5, mean = 0, sd = sqrt(phi))
paste(round((Q2_c2-1)*100,2), "%")

[1] "8.36 %"

**D.** La media y la desviación estándar cuando la inversión en TV y en radio es 150 y 25 están dadas por:

$$ \mu_k = g^{-1}(log(\mu_k)) = e^{\beta_0 + \beta_1 x_{1k}+ \beta_2 x_{2k} + \beta_3 x_{1k}x_{2k} }$$
$$ \sqrt{V(y_k)} = \sqrt{\frac{\hat{\phi}}{m_k} V(\mu_k)} =  \sqrt{\hat{\phi}} $$

In [10]:
#fit2$family$linkinv(fit2$coefficients %*% c(1, log(150),25, log(150)*25))
exp(fit2$coefficients %*% c(1, log(150),25, log(150)*25))

14.96538


In [11]:
sqrt(phi) #sqrt(fit2$deviance/fit2$df.residual)

[1] 0.4777535

**E.** $H_0:$ el parámetro de dispersión es constante.

In [12]:
vdtest(fit2)


             Score test for varying dispersion parameter

          Statistic =  5.75675 
 degrees of freedom =  3 
            p-value =  0.12406 



Como $p-value > 0.05$, entonces no hay evidencia estadísticamente significativa en contra de $H_0$

**F.** $D(y, \hat{\mu}) = (n-p)\hat{\phi}$

In [13]:
191*phi #deviance(fit2)

[1] 43.59544

# Punto 2
Estos datos corresponden a hospitalizaciones relacionadas con quemaduras que ocurrieron entre 1973 y 2007. El objetivo del análisis consiste en explicar las diferencias entre los pacientes en relación a su estado al momento del alta hospitalaria (en que $\texttt{alive}=1$ indica que el(la) paciente estaba vivo(a) al momento del alta hospitalaria y $\texttt{alive}=0$ indica lo contrario) a partir de diferencias entre los mismos con respecto a su edad en el momento de la admisión ($\texttt{age}$), en años, el área de la superficie quemada ($\texttt{tbsa}$), en porcentaje, y un indicador de la presencia de lesiones por inhalación ($\texttt{inh}\_\texttt{inj}$: “No” y “Yes”). Se ajusta a los datos un MLG usando R y la información resultante es la siguiente

In [14]:
burn1000 = aplore3::burn1000
attach(burn1000)

In [15]:
burn1000$alive = ifelse(burn1000$death == "Alive", 1, 0)

In [16]:
fit3 <- glm(alive ~ age*inh_inj + tbsa*inh_inj, family=binomial(logit), data = burn1000)
summary(fit3) #Residual deviance: 299.98


Call:
glm(formula = alive ~ age * inh_inj + tbsa * inh_inj, family = binomial(logit), 
    data = burn1000)

Deviance Residuals: 
     Min        1Q    Median        3Q       Max  
-2.64652   0.01159   0.04206   0.18489   2.86103  

Coefficients:
                Estimate Std. Error z value Pr(>|z|)    
(Intercept)     10.31738    1.03278   9.990  < 2e-16 ***
age             -0.11561    0.01352  -8.553  < 2e-16 ***
inh_injYes      -7.10938    1.25863  -5.649 1.62e-08 ***
tbsa            -0.11221    0.01439  -7.797 6.36e-15 ***
age:inh_injYes   0.08004    0.01709   4.683 2.82e-06 ***
inh_injYes:tbsa  0.05538    0.01799   3.079  0.00207 ** 
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 845.42  on 999  degrees of freedom
Residual deviance: 320.42  on 994  degrees of freedom
AIC: 332.42

Number of Fisher Scoring iterations: 8


In [17]:
fit4 <- glm(alive ~ age*inh_inj + tbsa*inh_inj, family=binomial(logit), data = burn1000[-c(357,697,913),])
summary(fit4) #Residual deviance: 299.98


Call:
glm(formula = alive ~ age * inh_inj + tbsa * inh_inj, family = binomial(logit), 
    data = burn1000[-c(357, 697, 913), ])

Deviance Residuals: 
     Min        1Q    Median        3Q       Max  
-2.70579   0.00756   0.03097   0.15572   2.35405  

Coefficients:
                Estimate Std. Error z value Pr(>|z|)    
(Intercept)     11.27847    1.18961   9.481  < 2e-16 ***
age             -0.12507    0.01521  -8.224  < 2e-16 ***
inh_injYes      -8.07047    1.39021  -5.805 6.43e-09 ***
tbsa            -0.13580    0.01757  -7.729 1.08e-14 ***
age:inh_injYes   0.08949    0.01846   4.849 1.24e-06 ***
inh_injYes:tbsa  0.07898    0.02062   3.831 0.000128 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 840.97  on 996  degrees of freedom
Residual deviance: 299.98  on 991  degrees of freedom
AIC: 311.98

Number of Fisher Scoring iterations: 8


In [18]:
set.seed(1234) # cuál es la semilla de los residuales ???
resis <- residuals2(fit4,type="quantile", plot.it = F)
resis[c(382,111)]

[1] -0.8169955  0.5672139

**A.** como la función de enlace es la canónica (logit), entonces $J(\beta)$ y $K(\beta)$ coinciden, lo cual, a su vez, implica que los algoritmos Newton-Raphson y scoring de Fisher también coinciden

In [19]:
-2*logLik(fit4) #fit4$deviance

'log Lik.' 299.9782 (df=6)

**B.** El chance de estar vivo dejando fijo el área de la superficie quemada

In [20]:
paste(round((exp(fit4$coefficients[2])-1)*100,3), "%")

[1] "-11.757 %"

In [21]:
# exp(fit4$coefficients[2]+fit4$coefficients[5]) # la estimación de B_5 y B_6 cambiaron de signo (???)
paste(round((exp(fit4$coefficients[2]-fit4$coefficients[5])-1)*100,2), "%")

[1] "-19.31 %"

**C.** El chance de estar vivo dejando fija la edad

In [22]:
paste(round((exp(fit4$coefficients[4])-1)*100,3), "%")

[1] "-12.698 %"

In [23]:
# exp(fit4$coefficients[4]+fit4$coefficients[5])
paste(round((exp(fit4$coefficients[4]-fit4$coefficients[6])-1)*100,3), "%")

[1] "-19.328 %"

**D.** El chance de estar vivo en pacientes con lesiones por inhalación ceteris paribus

In [24]:
# c(1,32,1,6,32,6): paciente de 32 años, con lesiones por inhalación y 6% de área de la superficie quemada
exp(fit4$coefficients[3]+fit4$coefficients[5]*32+fit4$coefficients[6]*6)
#exp(fit4$coefficients %*% c(1,32,1,6,32,6))/exp(fit4$coefficients %*% c(1,32,0,6,0,0))

inh_injYes 
0.008801886

como la razón de chances es menor que 1, indica que el chance de estar vivo en pacientes con lesiones por inhalación es menor que en pacientes sin este tipo de lesiones.

In [25]:
c("efecto"=exp(fit4$coefficients[3]+fit4$coefficients[5]*32+fit4$coefficients[6]*6),
  "o_tbsa"=exp(fit4$coefficients[3]+fit4$coefficients[5]*32+fit4$coefficients[6]*(6*2)),
  "o_age"=exp(fit4$coefficients[3]+fit4$coefficients[5]*(32*2)+fit4$coefficients[6]*6),
  "y_tbsa_age"=exp(fit4$coefficients[3]+fit4$coefficients[5]*(32*2)+fit4$coefficients[6]*(6*2))
 )

efecto.inh_injYes     o_tbsa.inh_injYes      o_age.inh_injYes 
          0.008801886           0.014137551           0.154282115 
y_tbsa_age.inh_injYes 
          0.247807273

Como $\hat{\beta}_4$ y $\hat{\beta}_5$ son coeficientes positivos, entonces el aumento de la edad y/o el área de superficie quemada contrarresta el efecto de las lesiones por inhalación sobre el chance de estar vivo. Caso contrario cuando los coeficientes son negativos (como en la salida del parcial) el efecto de las lesiones por inhalación sobre el chance de estar vivo se acentua con el aumento en edad y/o área de superficie quemada.

**E.** La probabilidad de estar vivo

$$
logit(\mu_k) = ln\left(\frac{\mu_k}{1-\mu_k}\right) = \boldsymbol{X_{k_{1 \times p+1}}} \boldsymbol{\beta_{(p+1) \times 1}}
$$

$$
\frac{\mu_k}{1-\mu_k} = \exp{\left(ln\left(\frac{\mu_k}{1-\mu_k}\right) \right)} = \boldsymbol{X_k}\boldsymbol{\beta} 
$$

$$
\frac{1}{\mu_k} - 1 = \left(\frac{\mu_k}{1-\mu_k}\right)^{-1} = \exp{(- \boldsymbol{X_k}\boldsymbol{\beta})}
$$

$$
\mu_k = \left(\frac{1}{\mu_k}\right)^{-1} = \frac{1}{\exp{(- \boldsymbol{X_k}\boldsymbol{\beta})} + 1} = \frac{\exp{(\boldsymbol{X_k}\boldsymbol{\beta})}}{\exp{( \boldsymbol{X_k}\boldsymbol{\beta})} + 1}
$$

In [26]:
# c(1,50,0,40,0,0): paciente de 50 años, sin lesiones por inhalación y 40% de área de la superficie quemada
#exp(fit4$coefficients%*%c(1,50,0,40,0,0)) # razón de chances (puede dar mayor a 1)
exp(fit4$coefficients%*%c(1,50,0,40,0,0))/(exp(fit4$coefficients%*%c(1,50,0,40,0,0))+1) #0.3995779

0.3995779


# Punto 3

Estos datos permiten calcular la incidencia (esto es, $10^r\times\texttt{cases}/\texttt{population}$ para algún $r$ entero y positivo) de cáncer de piel por rangos de edad ($\texttt{age}:$ “15-24”, “25-34”, “35-44”, “45-54”, “55-64”, “65-74”, “75-84” y “85+”) en las mujeres de las ciudades de Saint Paul ($\texttt{city}=$“St.Paul”) en Minnesota y Fort Worth ($\texttt{city}=$“Ft.Worth”) en Texas . Se espera que la incidencia de este tipo de cáncer sea mayor en Texas que en Minnesota ya que la exposición al sol es mayor en Texas. Se ajusta a los datos un MLG usando R y la información resultante es la siguiente

In [27]:
data(skincancer)
fit4 <- glm(cases ~ offset(log(population)) + city + age, family=poisson, data=skincancer)
summary(fit4)


Call:
glm(formula = cases ~ offset(log(population)) + city + age, family = poisson, 
    data = skincancer)

Deviance Residuals: 
     Min        1Q    Median        3Q       Max  
-1.50598  -0.48566   0.01639   0.36926   1.24763  

Coefficients:
             Estimate Std. Error z value Pr(>|z|)    
(Intercept)  -11.6576     0.4487 -25.980  < 2e-16 ***
cityFt.Worth   0.8039     0.0522  15.399  < 2e-16 ***
age25-34       2.6302     0.4675   5.627 1.84e-08 ***
age35-44       3.8474     0.4547   8.462  < 2e-16 ***
age45-54       4.5952     0.4510  10.188  < 2e-16 ***
age55-64       5.0873     0.4503  11.298  < 2e-16 ***
age65-74       5.6454     0.4497  12.552  < 2e-16 ***
age75-84       6.0585     0.4503  13.454  < 2e-16 ***
age85+         6.1742     0.4577  13.488  < 2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for poisson family taken to be 1)

    Null deviance: 2789.6810  on 15  degrees of freedom
Residual deviance:    8.2585  o

**A** En la distribución poisson tenemos que $\phi = 1$ y $m_k = 1$ con lo cual $Var(Y_k) = \mu$, entonces un aumento/disminución en le número esperado de casos de cáncer esta acompañada de un aumento/disminución en la varianza del número de casos de cáncer.

**B** La incidencia esperada de cáncer de piel en las mujeres de la ciudad de Forth Worth con edades en un rango especificado está dada por: $e^{\beta_2}$

In [28]:
exp(fit4$coefficients[2])

cityFt.Worth 
    2.234234

**C** La incidencia esperada de cáncer de piel en las mujeres de 25-35 años en la ciudad especificada respecto a las mujeres en la misma ciudad con edades en el rango de 15-24 años está dada por: $e^{\beta_3}$

In [29]:
exp(fit4$coefficients[3])

age25-34 
13.87656

**D** en este caso el modelo puede ser escrito: $log(\mu) = \boldsymbol{X_{k_{1 \times p+1}}} \boldsymbol{\beta_{(p+1) \times 1}} + log(n_{FW})$ de donde tenemos que:
$$\mu = \exp(\boldsymbol{X} \boldsymbol{\beta} + log(n_{FW}))$$

In [30]:
n_FW_25a34 = 200000
exp(fit4$coefficients[1]+fit4$coefficients[2]+fit4$coefficients[3] + log(n_FW_25a34))

(Intercept) 
   53.65433

**E** En este caso tenemos que $\phi = 1 $, mientras que:
$$AIC = -2 \ell(\hat{\boldsymbol{\beta}}) + 2p \quad\text{con lo cual} -2 \ell(\hat{\boldsymbol{\beta}}) = AIC -2p$$
$$ BIC =  -2 \ell(\hat{\boldsymbol{\beta}}) + log(n)p = (AIC -2p) + log(n)p$$

In [31]:
p = ncol(model.matrix(fit4))
n = nrow(model.matrix(fit4))

round(fit4$aic - 2*p, 1) # -2*logLik(fit4)
round(fit4$aic - 2*p + log(n)*p,3)

[1] 102.5

[1] 127.457

**F** A partir de la salida del test de razón de verosimilitudes se concluye que dejando fijo el rango de edad, la incidencia esperada del cáncer de piel depende de la ciudad. Análogamente se llega a la misma conclusión al realizar el test de Wald, por último a partir del estadístico del test de razón de verosilmilitudes podemos despejar la devianza del modelo $\texttt{fit42}$

In [32]:
# estadístico de Wald
fit4$coefficients[2]^2/vcov(fit4)[2,2] 
pchisq(fit4$coefficients[2]^2/vcov(fit4)[2,2], df = 1, lower.tail = F)

cityFt.Worth 
    237.1276

cityFt.Worth 
1.663497e-53

In [33]:
fit42 <- glm(cases ~ age, offset=log(population), family=poisson(log), data=skincancer)
anova2(fit42,fit4,test="lr")


  Likelihood-ratio test 

Model 1 :  cases ~ age 
Model 2 :  cases ~ offset(log(population)) + city + age 

        Chi    df  Pr(Chisq>)    
1 vs 2 258.48   1   < 2.2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1


In [34]:
round(258.48+fit4$deviance,4)

[1] 266.7385

# Punto 4

Estos datos describen las características principales de 392 automóviles. El objetivo del análisis consiste en explicar las diferencias entre los automóviles con respecto a su rendimiento ($\texttt{mpg}$), en millas por galón, usando la potencia de su motor, en número de caballos de fuerza ($\texttt{horsepower}$), y su peso ($\texttt{weight}$), en miles de libras. Se ajusta a los datos un MLG usando R y la información resultante es la siguiente:

In [35]:
Auto <- ISLR::Auto
Auto$weight = Auto$weight/1000

In [36]:
fit5 <- glm(mpg ~ horsepower + weight + horsepower*weight, family=Gamma(log), data=Auto)
summary(fit5)


Call:
glm(formula = mpg ~ horsepower + weight + horsepower * weight, 
    family = Gamma(log), data = Auto)

Deviance Residuals: 
     Min        1Q    Median        3Q       Max  
-0.42249  -0.10530  -0.01538   0.09485   0.54643  

Coefficients:
                    Estimate Std. Error t value Pr(>|t|)    
(Intercept)        4.5317628  0.0929819  48.738  < 2e-16 ***
horsepower        -0.0072027  0.0010826  -6.653 9.79e-11 ***
weight            -0.3620461  0.0307093 -11.789  < 2e-16 ***
horsepower:weight  0.0012047  0.0002639   4.565 6.70e-06 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for Gamma family taken to be 0.02432275)

    Null deviance: 44.2046  on 391  degrees of freedom
Residual deviance:  9.1561  on 388  degrees of freedom
AIC: 2080.4

Number of Fisher Scoring iterations: 4


**A** En la distribución $gamma(\nu, \lambda)$ tenemos que $\phi = 1/\nu$ y $m_k = 1$ con lo cual $Var(Y_k) = \phi \mu^2$, entonces un aumento/disminución en el rendimiento está acompañado de un aumento/disminución en la desviación estándar del rendimiento.

**B** Al realizar el test de Wald se concluye que el efecto del número de caballos de fuerza sobre el rendimiento esperado del vehículo depende de su peso:

In [37]:
# estadístico de Wald
sqrt(fit5$coefficients[4]^2/vcov(fit5)[4,4])
pt(sqrt(fit5$coefficients[4]^2/vcov(fit5)[4,4]), df= 1, lower.tail = F)

horsepower:weight 
         4.565437

horsepower:weight 
       0.06863768

**C** el rendimiento *mediano* dejando fijo el número de caballos de fuerza se reduce a medida que su peso aumenta

In [38]:
paste(round((exp(fit5$coefficients[3]*1+fit5$coefficients[4]*50*1)-1)*100,2), "%")

[1] "-26.05 %"

In [39]:
paste(round((exp(fit5$coefficients[3]*1+fit5$coefficients[4]*200*1)-1)*100,2), "%")

[1] "-11.41 %"

**D** el rendimiento *mediano* dejando fijo el peso se reduce a medida que la fuerza de su motor aumenta

In [40]:
paste(round((exp(fit5$coefficients[2]*20+fit5$coefficients[4]*20*2)-1)*100,2), "%")

[1] "-9.14 %"

In [41]:
paste(round((exp(fit5$coefficients[2]*20+fit5$coefficients[4]*20*4)-1)*100,2), "%")

[1] "-4.66 %"

**E** La media y la desviación estándar del rendimiento de un vehículo con 100 caballos de fuerza y 3000 libras de peso están dadas por:

In [42]:
# media
round(exp(fit5$coefficients[1]+fit5$coefficients[2]*100+fit5$coefficients[3]*3+fit5$coefficients[4]*100*3),3)

(Intercept) 
     21.906

In [43]:
# desviación estándar
phi = summary(fit5)$dispersion
round(sqrt(phi*exp(fit5$coefficients[1]+fit5$coefficients[2]*100+fit5$coefficients[3]*3+fit5$coefficients[4]*100*3)^2),3)

(Intercept) 
      3.416

**F** A partir de la salida de R tenemos los valores de $\phi$ y $D(\boldsymbol{\hat{\beta}})$, mientras que el $R^2$ y $R^2 \quad\text{ajustado}$ están dados por:

$$
R^2_{ajustado} = 1 − \frac{D(y, \hat{\mu})}{D(y, \hat{\mu}^0)}
$$

$$
R^2_{ajustado} = 1 − \frac{D(y, \hat{\mu})/(n-p)}{D(y, \hat{\mu}^0)/(n-1)}
$$

In [44]:
phi
fit5$deviance

[1] 0.02432275

[1] 9.156077

In [45]:
round(1-(fit5$deviance/fit5$null.deviance),5) # R^2
round(1-((fit5$deviance/fit5$df.residual)/(fit5$null.deviance/fit5$df.null)),5) # R^2 ajustado

[1] 0.79287

[1] 0.79127

In [46]:
sessionInfo()

R version 3.6.1 (2019-07-05)
Platform: x86_64-conda_cos6-linux-gnu (64-bit)
Running under: Ubuntu 20.04.5 LTS

Matrix products: default
BLAS/LAPACK: /home/fm/anaconda3/envs/R/lib/R/lib/libRblas.so

locale:
 [1] LC_CTYPE=en_US.UTF-8       LC_NUMERIC=C              
 [3] LC_TIME=es_CO.UTF-8        LC_COLLATE=en_US.UTF-8    
 [5] LC_MONETARY=es_CO.UTF-8    LC_MESSAGES=en_US.UTF-8   
 [7] LC_PAPER=es_CO.UTF-8       LC_NAME=C                 
 [9] LC_ADDRESS=C               LC_TELEPHONE=C            
[11] LC_MEASUREMENT=es_CO.UTF-8 LC_IDENTIFICATION=C       

attached base packages:
[1] stats     graphics  grDevices utils     datasets  methods   base     

other attached packages:
[1] glmtoolbox_0.1.4   Formula_1.2-4      Rfast_2.0.6        RcppZiggurat_0.1.6
[5] Rcpp_1.0.1        

loaded via a namespace (and not attached):
 [1] digest_0.6.18     crayon_1.3.4      IRdisplay_0.7.0   repr_0.19.2      
 [5] jsonlite_1.6      evaluate_0.13     uuid_0.1-2        ISLR_1.4         
 [9] IRkernel_